In [1]:
import pandas as pd
import numpy as np

fund_list = pd.read_csv('data/temp/fund_list_2022-12-05.csv')

In [10]:
type_dict = {1: 'All', 2: 'Funds under Chapter 7 of the Code on Unit Trusts and Mutual Funds (excluding funds with guaranteed features)', 3: 'Unit Portfolio Management Funds (Ch 8.1)', 4: 'Money Market Funds (Ch 8.2)', 5: 'Futures and Options Funds (Ch 8.4A)', 6: 'Funds with guaranteed features', 7: 'Index Funds (Ch 8.6)', 8: 'Hedge Funds (Ch 8.7)', 9: 'Structured Funds (Ch 8.8)', 10: 'Funds that invest extensively in financial derivative instruments (Ch 8.9)', 11: 'UCITS Funds', 12: 'Recognised Mainland Funds', 13: 'Recognised Malaysian Islamic Funds', 14: 'Recognised Luxembourg UCITS Funds'}
for n in range(2,15):
    temp = fund_list['type'].apply(lambda x: x.split(';')[n-2])
    print(type_dict[n], temp.apply(lambda x: 1 if x != '0' else 0).sum())

Funds under Chapter 7 of the Code on Unit Trusts and Mutual Funds (excluding funds with guaranteed features) 409
Unit Portfolio Management Funds (Ch 8.1) 0
Money Market Funds (Ch 8.2) 52
Futures and Options Funds (Ch 8.4A) 0
Funds with guaranteed features 0
Index Funds (Ch 8.6) 45
Hedge Funds (Ch 8.7) 0
Structured Funds (Ch 8.8) 2
Funds that invest extensively in financial derivative instruments (Ch 8.9) 0
UCITS Funds 1201
Recognised Mainland Funds 0
Recognised Malaysian Islamic Funds 0
Recognised Luxembourg UCITS Funds 4


In [5]:
fee = pd.read_csv('data/fee_table_1.csv')
# num of unique id
len(fee['id'].unique())
# delete duplicated id
fee = fee.drop_duplicates(subset=['id'], keep='first')
fee.columns = ['id_subfund', 'subscription', 'switching', 'redemption', 'management',
       'trustee', 'performance', 'admin']
# keep only funds with id in fee
fund_list = fund_list[fund_list['id_subfund'].isin(fee['id_subfund'])]
fund_list = pd.merge(fund_list, fee, on='id_subfund', how='left')

In [13]:
# data analysis
fund_list['year'] = fund_list['date_auth'].apply(lambda x: int(x.split('-')[0]))
# set time window
df = fund_list.loc[fund_list['year'] >= 2019]

In [14]:
df.to_csv('data/temp/output_2022-12-05.csv', index=False)